<a href="https://colab.research.google.com/github/mshumer/ai-researcher/blob/main/Claude_Researcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://github.com/joriskalz/ai-researcher.git

In [ ]:
!pip install google-api-python-client
!pip install openai

from googleapiclient.discovery import build
import concurrent.futures
import ast
import os
import re
from openai import OpenAI
import json


# ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Replace with your OpenAI API key
GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)


In [ ]:
!pip install google-api-python-client
!pip install openai

from googleapiclient.discovery import build
import concurrent.futures
import ast
import os
import re
from openai import OpenAI
import json


# ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Replace with your OpenAI API key
GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(prompt, model="gpt-4-0125-preview", max_tokens=2000, temperature=0.7, n=1, stop=None, presence_penalty=0, frequency_penalty=0.1):
    system_prompt = "You are a world-class researcher. Analyze the given information and generate a well-structured report."
    try:
        messages = [
            {"role": "system", "content": f"{system_prompt}"},
            {"role": "user", "content": prompt},
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            n=n,
            stop=stop,
            presence_penalty=presence_penalty,
            frequency_penalty=frequency_penalty,
        )
        if response.choices:
            response_text = response.choices[0].message.content.strip()
            print(remove_first_line(response_text.strip()))
            return remove_first_line(response_text.strip())
        else:
            raise Exception("No text generated by the API.")
    except Exception as e:
        raise Exception(f"Request failed: {e}")

def search_web(search_term, api_key, cse_id):
    service = build("customsearch", "v1", developerKey=api_key)
    result = service.cse().list(q=search_term, cx=cse_id).execute()
    # Parse the JSON data
    parsed_data = json.loads(json.dumps(result))
    # Extract relevant text from the items
    relevant_text = []
    for item in parsed_data['items']:
        title = item['title']
        snippet = item.get('snippet', '')  # Use get() to handle missing 'snippet' key
        relevant_text.append(f"Title: {title}\nSnippet: {snippet}\n")
    # Print the relevant text
    print("\n".join(relevant_text))
    return relevant_text

def generate_subtopic_report(research_topic, subtopic):
    search_data = []
    all_queries = []
    search_cache = {}
    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 5 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
 
    initial_queries_response = generate_text(initial_queries_prompt)
    if initial_queries_response.startswith('[') and ']' in initial_queries_response:
        try:
            initial_queries = ast.literal_eval(initial_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping initial queries.")
            initial_queries = []
    else:
        print("Error: Search query format not found. Skipping initial queries.")
        initial_queries = []
   
    print(initial_queries)
    all_queries.extend(initial_queries)

    def search_and_cache(query):
        if query in search_cache:
            return search_cache[query]
        else:
            search_results = search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)
            search_cache[query] = search_results
            return search_results

    with concurrent.futures.ThreadPoolExecutor() as executor:
        search_results = list(executor.map(search_and_cache, initial_queries))
        search_data.extend(search_results)
 
    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}' of the research topic '{research_topic}':\n\n{str(search_data)}"
    report = generate_text(report_prompt, max_tokens=4000)

    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt = f"Analyze the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' and identify areas that need more detail or further information:\n\n{report}\n\n---\n\nHere are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\nGenerate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
   
    additional_queries_response = generate_text(analysis_prompt)
    if additional_queries_response.startswith('[') and ']' in additional_queries_response:
        try:
            additional_queries = ast.literal_eval(additional_queries_response)
        except SyntaxError:
            print("Error: Invalid search query format. Skipping additional queries.")
            additional_queries = []
    else:
        print("Error: Search query format not found. Skipping additional queries.")
        additional_queries = []
   
    all_queries.extend(additional_queries)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        additional_search_results = list(executor.map(search_and_cache, additional_queries))
        search_data.extend(additional_search_results)

    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt = f"Update the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data:\n\n{str(additional_search_results)}\n\n---\n\nGenerate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
    report = generate_text(update_prompt, max_tokens=4000)

    print(f"Final report generated for subtopic: {subtopic}!")
    return report

def generate_comprehensive_report(research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt = f"Generate a comprehensive report on the research topic '{research_topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the research topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report = generate_text(comprehensive_report_prompt, model="gpt-4-0125-preview", max_tokens=4000)
    print("Comprehensive report generated!")
    return comprehensive_report

# User input
research_topic = input("Enter the research topic: ")

# Generate subtopic checklist
subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist in a Python-parseable list. Return nothing but the list. Do so in one line. Maximum five sub-topics. Start your response with [\""
subtopic_checklist = ast.literal_eval('[' + generate_text(subtopic_checklist_prompt).split('[')[1])
print(f"Subtopic Checklist: {subtopic_checklist}")

# Generate reports for each subtopic
subtopic_reports = []
for subtopic in subtopic_checklist:
    subtopic_report = generate_subtopic_report(research_topic, subtopic)
    subtopic_reports.append(subtopic_report)

# Combine subtopic reports into a comprehensive report
comprehensive_report = generate_comprehensive_report(research_topic, "\n\n".join(subtopic_reports))

# Save the comprehensive report to a file
with open("comprehensive_report.txt", "w") as file:
    file.write(comprehensive_report)

print("Comprehensive report saved as 'comprehensive_report.txt'.")

# Comprehensive Report on Bipolar Disorder

## Table of Contents

1. [Introduction](#Introduction)
2. [Definition and Types of Bipolar Disorder](#Definition-and-Types-of-Bipolar-Disorder)
    - 2.1. [Overview](#Overview)
    - 2.2. [Symptoms and Signs](#Symptoms-and-Signs)
    - 2.3. [Types of Bipolar Disorder](#Types-of-Bipolar-Disorder)
3. [Causes and Risk Factors](#Causes-and-Risk-Factors)
    - 3.1. [Genetic Factors](#Genetic-Factors)
    - 3.2. [Neurochemical Factors](#Neurochemical-Factors)
    - 3.3. [Environmental Triggers](#Environmental-Triggers)
    - 3.4. [Brain Structure and Function](#Brain-Structure-and-Function)
    - 3.5. [Hormonal Influences](#Hormonal-Influences)
    - 3.6. [Lifestyle Factors](#Lifestyle-Factors)
4. [Symptoms and Diagnosis](#Symptoms-and-Diagnosis)
    - 4.1. [Diagnostic Process](#Diagnostic-Process)
    - 4.2. [Advancements in Understanding and Diagnosing Bipolar Disorder](#Advancements-in-Understanding-and-Diagnosing-Bipolar-Disorder)
5. [Treatment Options](#Treatment-Options)
    - 5.1. [Pharmacological Treatments](#Pharmacological-Treatments)
    - 5.2. [Psychotherapeutic Interventions](#Psychotherapeutic-Interventions)
    - 5.3. [Lifestyle Interventions and Adjunctive Therapies](#Lifestyle-Interventions-and-Adjunctive-Therapies)
    - 5.4. [Emerging Treatment Options](#Emerging-Treatment-Options)
6. [Impact on Daily Life](#Impact-on-Daily-Life)
    - 6.1. [Work and Education](#Work-and-Education)
    - 6.2. [Relationships](#Relationships)
    - 6.3. [Self-Management and Lifestyle Adjustments](#Self-Management-and-Lifestyle-Adjustments)
    - 6.4. [Challenges and Coping Strategies](#Challenges-and-Coping-Strategies)
7. [Conclusion and Recommendations](#Conclusion-and-Recommendations)
8. [References](#References)

---

## Introduction

Bipolar disorder, previously known as manic depression, is a mental health condition marked by extreme mood swings, including manic or hypomanic episodes (emotional highs) and depressive episodes (emotional lows). This report provides a comprehensive analysis of bipolar disorder, covering its definition, types, causes, risk factors, symptoms, diagnosis, treatment options, and the impact on daily life.

## Definition and Types of Bipolar Disorder

### Overview

Bipolar disorder leads to significant mood fluctuations beyond normal ups and downs, affecting mood, energy, activity levels, concentration, and daily task performance.

### Symptoms and Signs

- **Mania/Hypomania**: Euphoria, inflated self-esteem, decreased need for sleep.
- **Depression**: Sadness, hopelessness, loss of energy.

### Types of Bipolar Disorder

1. **Bipolar I Disorder**: Characterized by manic episodes lasting at least 7 days or severe manic symptoms requiring immediate hospital care.
2. **Bipolar II Disorder**: Defined by patterns of depressive episodes and hypomanic episodes without the full-blown manic episodes seen in Bipolar I Disorder.
3. **Cyclothymic Disorder (Cyclothymia)**: Milder mood swings over two years or more.
4. **Rapid Cycling Bipolar Disorder**: Four or more mood episodes within a year.
5. **Bipolar Disorder with Mixed Features**: Symptoms of mania/hypomania and depression simultaneously.
6. **Bipolar Disorder with Seasonal Pattern**: Mood changes influenced by seasonal variations.

## Causes and Risk Factors

### Genetic Factors

A strong genetic component is evidenced by family history and twin studies.

### Neurochemical Factors

Imbalances of neurotransmitters like serotonin, norepinephrine, and dopamine are crucial.

### Environmental Triggers

Childhood trauma and stressful life events are significant triggers.

### Brain Structure and Function

Alterations contribute to the risk and manifestation of bipolar disorder.

### Hormonal Influences

Hormonal imbalances may play a role in mood episode variations.

### Lifestyle Factors

Poor diet, lack of physical activity, smoking, and alcohol use may exacerbate symptoms.

## Symptoms and Diagnosis

### Diagnostic Process

Diagnosing bipolar disorder is complex due to overlapping symptoms with other psychiatric conditions.

### Advancements in Understanding and Diagnosing Bipolar Disorder

Genomic studies have identified genes associated with bipolar disorder, offering pathways toward more objective diagnostic criteria.

## Treatment Options

### Pharmacological Treatments

Includes mood stabilizers (Lithium, Anticonvulsants), antipsychotics, antidepressants, Ketamine, and Lumateperone.

### Psychotherapeutic Interventions

CBT, psychoeducation, family-focused therapy, and interpersonal and social rhythm therapy are effective.

### Lifestyle Interventions and Adjunctive Therapies

Exercise, dietary modifications, sleep hygiene, mindfulness, and stress management complement treatment.

### Emerging Treatment Options

ECT, Theta-burst stimulation, Neurofeedback, and novel pharmacological targets like CaMKK2 enzyme inhibitors show promise.

## Impact on Daily Life

### Work and Education

Challenges with financial decision-making during manic episodes highlight the need for structured financial management support.

### Relationships

BD can lead to family dysfunction; therapy is recommended to improve communication and coping strategies.

### Self-Management and Lifestyle Adjustments

Digital health technologies help track mood swings, medication adherence, and offer reminders for routine activities.

### Challenges and Coping Strategies

Enhanced resources and support networks are necessary for families coping with BD.

## Conclusion and Recommendations

Bipolar disorder is a complex condition requiring a multifaceted treatment approach and support system to manage its diverse symptoms effectively. Recommendations include the development of financial management programs, family support initiatives, integration of digital health technologies in treatment plans, and increased education on BD management.

## References

The referenced materials include data from the National Institute of Mental Health (NIMH), Psychiatry.org, Mayo Clinic, Mind UK, Substance Abuse and Mental Health Services Administration (SAMHSA), National Alliance on Mental Illness (NAMI), MedlinePlus, American Psychiatric Association (DSM�5), NHS, ScienceDaily, PMC (PubMed Central), Harvard Medical School, NYU Langone Health, National Institutes of Health (NIH), Johns Hopkins Medicine, Clearview, Casa Recovery among others up to April 2023.